### Delta Lake Reading Databricks Datasets Already Provided

[Delta Lake Tutorial](https://docs.databricks.com/en/delta/tutorial.html#language-sql): 
This tutorial introduces common Delta Lake operations on Databricks.
Delta Lake is the optimized storage layer that provides the foundation for storing data and tables in the Databricks Lakehouse Platform. 
Delta Lake is open source software that extends Parquet data files with a file-based transaction log for ACID transactions and scalable metadata handling.

In [0]:
%python
dbutils.fs.ls("/databricks-datasets/flights/")

[FileInfo(path='dbfs:/databricks-datasets/flights/README.md', name='README.md', size=412, modificationTime=1457766852000),
 FileInfo(path='dbfs:/databricks-datasets/flights/airport-codes-na.txt', name='airport-codes-na.txt', size=11411, modificationTime=1457749605000),
 FileInfo(path='dbfs:/databricks-datasets/flights/departuredelays.csv', name='departuredelays.csv', size=33396236, modificationTime=1457749605000)]

In [0]:
%sql
drop table if exists delayflights;
create table if not exists delayflights
as select * 
from read_files("/databricks-datasets/flights/departuredelays.csv")

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE DETAIL delayflights;

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,6ffc8859-ec0d-4550-869b-40891bd3dd75,spark_catalog.default.delayflights,null,dbfs:/user/hive/warehouse/delayflights,2024-01-27T15:24:57.685Z,2024-01-27T15:25:35Z,List(),List(),8,6774798,Map(),1,2,"List(appendOnly, invariants)",Map()


### Browse Databricks Datasets - https://docs.databricks.com/en/discover/databricks-datasets.html#databricks-datasets-databricks-datasets 

In [0]:
%python
dbutils.fs.ls("/databricks-datasets/learning-spark-v2/people/")

[FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/.DS_Store', name='.DS_Store', size=6148, modificationTime=1602174521000),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/README.md', name='README.md', size=215, modificationTime=1587070414000),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.delta/', name='people-10m.delta/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.parquet/', name='people-10m.parquet/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-with-header-10m.csv.bzip/', name='people-with-header-10m.csv.bzip/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-with-header-10m.txt', name='people-with-header-10m.txt', size=608145966, modificationTime=1587070596000),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-with-

### Lets read in a Delta table and save it in a new table called people_10m. 
Notice the .delta below. 

In [0]:
%sql
DROP TABLE IF EXISTS people_10m;

CREATE TABLE IF NOT EXISTS people_10m
AS SELECT * FROM delta.`/databricks-datasets/learning-spark-v2/people/people-10m.delta`;

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE DETAIL people_10m;

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,5e9eff92-4f02-43ac-b200-4b9c1b1f102f,spark_catalog.default.people_10m,null,dbfs:/user/hive/warehouse/people_10m,2024-01-27T16:18:41.993Z,2024-01-27T16:19:51Z,List(),List(),8,236921468,Map(),1,2,"List(appendOnly, invariants)",Map()


### Go to Catalog Now
dbfs:/user/hive/warehouse/people_10m - the location is in the Delta Wearhouse.

But also notice we created a new table too in Default


In [0]:
%sql
select count(*) from people_10m

count(1)
10000000


## Upsert to a Table
To merge a set of updates and insertions into an existing Delta table, you use the [MERGE INTO](https://docs.databricks.com/en/sql/language-manual/delta-merge-into.html) statement

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW people_updates (
  id, firstName, middleName, lastName, gender, birthDate, ssn, salary
) AS VALUES
  (9999998, 'Hulk', 'Bruce', 'Banner', 'M', '1992-09-17T04:00:00.000+0000', '953-38-9452', 55250),
  (9999999, 'Batman', 'Bruce', 'Wayne', 'M', '1984-05-22T04:00:00.000+0000', '906-51-2137', 48500),
  (10000000, 'Thor', 'Chris', 'Hemsworth', 'M', '1968-07-22T04:00:00.000+0000', '988-61-6247', 90000),
  (20000001, 'Spiderman', '', 'Parker', 'M', '1978-01-14T04:00:00.000+000', '345-67-8901', 55500),
  (20000002, 'Captain', '', 'America', 'M', '1982-10-29T01:00:00.000+000', '456-78-9012', 98250),
  (20000003, 'Wonder', '', 'Woman', 'F', '1981-06-25T04:00:00.000+000', '567-89-0123', 89900);

MERGE INTO people_10m
USING people_updates
ON people_10m.id = people_updates.id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *;

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 52.0 failed 1 times, most recent failure: Lost task 3.0 in stage 52.0 (TID 120) (ip-10-172-243-241.us-west-2.compute.internal executor driver): org.apache.spark.SparkDateTimeException: [CAST_INVALID_INPUT] The value '1978-01-14T04:00:00.000+000' of the type "STRING" cannot be cast to "TIMESTAMP" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. If necessary set "spark.sql.ansi.enabled" to "false" to bypass this error. SQLSTATE: 22018
== SQL (line 1, position 1) ==
MERGE INTO people_10m
^^^^^^^^^^^^^^^^^^^^^
USING people_updates
^^^^^^^^^^^^^^^^^^^^
ON people_10m.id = people_updates.id
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
WHEN MATCHED THEN UPDATE SET *
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
WHEN NOT MATCHED THEN INSERT *
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

	at org.apache.spark.sql.errors.ExecutionErrors.invalidInputInCastToDatetimeErrorInternal(ExecutionErrors.scala:114)
	at org.apache.spark.sql.errors.ExecutionErrors.invalidInputInCastToDatetimeErrorInternal$(ExecutionErrors.scala:101)
	at org.apache.spark.sql.errors.ExecutionErrors$.invalidInputInCastToDatetimeErrorInternal(ExecutionErrors.scala:230)
	at org.apache.spark.sql.errors.ExecutionErrors.invalidInputInCastToDatetimeError(ExecutionErrors.scala:91)
	at org.apache.spark.sql.errors.ExecutionErrors.invalidInputInCastToDatetimeError$(ExecutionErrors.scala:87)
	at org.apache.spark.sql.errors.ExecutionErrors$.invalidInputInCastToDatetimeError(ExecutionErrors.scala:230)
	at org.apache.spark.sql.catalyst.util.SparkDateTimeUtils.$anonfun$stringToTimestampAnsi$1(SparkDateTimeUtils.scala:588)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.catalyst.util.SparkDateTimeUtils.stringToTimestampAnsi(SparkDateTimeUtils.scala:588)
	at org.apache.spark.sql.catalyst.util.SparkDateTimeUtils.stringToTimestampAnsi$(SparkDateTimeUtils.scala:583)
	at org.apache.spark.sql.catalyst.util.DateTimeUtils$.stringToTimestampAnsi(DateTimeUtils.scala:40)
	at org.apache.spark.sql.catalyst.util.DateTimeUtils.stringToTimestampAnsi(DateTimeUtils.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage3.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.hasNext(WholeStageCodegenEvaluatorFactory.scala:43)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:525)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFiles.scala:116)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:931)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:931)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:60)
	at org.apache.spark.rdd.RDD.$anonfun$computeOrReadCheckpoint$1(RDD.scala:407)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:404)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:371)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$3(ResultTask.scala:82)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$1(ResultTask.scala:82)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:62)
	at org.apache.spark.TaskContext.runTaskWithListeners(TaskContext.scala:196)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:181)
	at org.apache.spark.schedu

In [0]:
%sql
DESCRIBE people_10m;

col_name,data_type,comment
id,int,null
firstName,string,null
middleName,string,null
lastName,string,null
gender,string,null
birthDate,timestamp,null
ssn,string,null
salary,int,null


In [0]:
%sql
SHOW COLUMNS FROM people_10m;

col_name
id
firstName
middleName
lastName
gender
birthDate
ssn
salary


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW people_updates (
  id, firstName, middleName, lastName, gender, birthDate, ssn, salary
) AS VALUES
  (9999998, 'Hulk', 'Bruce', 'Banner', 'M', '1992-09-17T04:00:00.000+0000', '953-38-9452', 55250),
  (9999999, 'Batman', 'Bruce', 'Wayne', 'M', '1984-05-22T04:00:00.000+0000', '906-51-2137', 48500),
  (10000000, 'Thor', 'Chris', 'Hemsworth', 'M', '1968-07-22T04:00:00.000+0000', '988-61-6247', 90000),
  (20000001, 'Spiderman', '', 'Parker', 'M', '1978-01-14T04:00:00.000+000', '345-67-8901', 55500),
  (20000002, 'Captain', '', 'America', 'M', '1982-10-29T01:00:00.000+000', '456-78-9012', 98250),
  (20000003, 'Wonder', '', 'Woman', 'F', '1981-06-25T04:00:00.000+000', '567-89-0123', 89900);

MERGE INTO people_10m
USING (
  SELECT id, firstName, middleName, lastName, gender, ssn, salary, try_cast(birthDate AS timestamp) AS birthDate
  FROM people_updates
) AS updates
ON people_10m.id = updates.id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
6,3,0,3


## Read a table

In [0]:
%sql
select * from people_10m limit 10;

id,firstName,middleName,lastName,gender,birthDate,ssn,salary
5016568,Enrique,Emmett,Carvil,M,1960-01-17T05:00:00Z,909-88-7612,62241
5016569,Jordan,Alva,Penk,M,1979-05-02T04:00:00Z,936-39-5888,74778
5016570,Leo,Merlin,Conkay,M,1981-05-21T04:00:00Z,919-71-2948,82321
5016571,Bernard,Wiley,Thackham,M,1983-10-12T04:00:00Z,976-38-5505,72797
5016572,Devin,Loyd,Gipp,M,1990-01-20T05:00:00Z,934-44-9546,99538
5016573,Howard,Wade,Cokly,M,1979-04-10T05:00:00Z,901-17-5955,39998
5016574,Clyde,Anibal,Silvester,M,1964-02-03T05:00:00Z,992-74-4926,88749
5016575,Craig,Doug,Dohmer,M,1971-10-28T04:00:00Z,943-46-5612,75135
5016576,Eddie,Mikel,Handyside,M,1962-01-07T05:00:00Z,916-66-9845,67990
5016577,Burton,Scotty,Broggini,M,1968-02-16T05:00:00Z,973-55-2258,64515


In [0]:
%sql
select * from delta.`dbfs:/user/hive/warehouse/people_10m` limit 10;

id,firstName,middleName,lastName,gender,birthDate,ssn,salary
5016568,Enrique,Emmett,Carvil,M,1960-01-17T05:00:00Z,909-88-7612,62241
5016569,Jordan,Alva,Penk,M,1979-05-02T04:00:00Z,936-39-5888,74778
5016570,Leo,Merlin,Conkay,M,1981-05-21T04:00:00Z,919-71-2948,82321
5016571,Bernard,Wiley,Thackham,M,1983-10-12T04:00:00Z,976-38-5505,72797
5016572,Devin,Loyd,Gipp,M,1990-01-20T05:00:00Z,934-44-9546,99538
5016573,Howard,Wade,Cokly,M,1979-04-10T05:00:00Z,901-17-5955,39998
5016574,Clyde,Anibal,Silvester,M,1964-02-03T05:00:00Z,992-74-4926,88749
5016575,Craig,Doug,Dohmer,M,1971-10-28T04:00:00Z,943-46-5612,75135
5016576,Eddie,Mikel,Handyside,M,1962-01-07T05:00:00Z,916-66-9845,67990
5016577,Burton,Scotty,Broggini,M,1968-02-16T05:00:00Z,973-55-2258,64515


## Update a table

In [0]:
%sql
UPDATE people_10m SET gender = 'Superhero' WHERE gender = 'Female';
UPDATE people_10m SET gender = 'Eye Candy' WHERE gender = 'Male';

num_affected_rows
4812700


In [0]:
%sql
select * from people_10m limit 50;

id,firstName,middleName,lastName,gender,birthDate,ssn,salary
1267751,Hester,Sherie,Bonome,Superhero,1999-09-13T04:00:00Z,943-76-3317,81098
1267752,Shanae,Lucie,Ahlin,Superhero,1980-02-13T05:00:00Z,987-96-3819,87092
1267753,Vasiliki,Candelaria,Gipps,Superhero,1960-06-20T04:00:00Z,987-24-9098,90477
1267754,Sue,Alba,Gullefant,Superhero,1978-10-14T04:00:00Z,995-81-4875,95425
1267755,Clemmie,Lisabeth,Clayfield,Superhero,1990-09-30T04:00:00Z,905-20-1783,65913
1267756,Elza,Ayako,Boynton,Superhero,1984-04-01T05:00:00Z,935-85-5288,31122
1267757,Berniece,Shelley,Cutler,Superhero,1999-07-09T04:00:00Z,980-55-6967,74981
1267758,Gwyneth,Gilda,Futter,Superhero,1983-07-08T04:00:00Z,969-50-3671,104619
1267759,Marine,Kayce,Irving,Superhero,1963-06-07T04:00:00Z,924-81-6464,68535
1267760,Patience,Mellie,Goodanew,Superhero,1957-07-19T04:00:00Z,995-95-3098,104723


In [0]:
%sql
select * from people_10m where gender = 'Eye Candy' limit 20;

id,firstName,middleName,lastName,gender,birthDate,ssn,salary
5000001,Jasper,Gil,Neubigin,Eye Candy,1961-02-20T05:00:00Z,900-26-1632,71908
5000002,Robert,Lindsay,Kipling,Eye Candy,1986-01-02T05:00:00Z,971-15-9711,73156
5000003,Abdul,Mickey,Pachta,Eye Candy,1956-01-28T05:00:00Z,935-93-3364,71366
5000004,Earle,Logan,Gieraths,Eye Candy,1958-04-17T05:00:00Z,995-20-3583,82976
5000005,Mickey,August,Wotton,Eye Candy,1976-08-29T04:00:00Z,666-63-8912,53965
5000006,Theron,Theo,Aplin,Eye Candy,1995-03-16T05:00:00Z,902-85-9597,92844
5000007,Brice,Jefferson,Charge,Eye Candy,1973-07-17T04:00:00Z,983-50-9434,66020
5000008,Valentin,Aubrey,Yakunin,Eye Candy,1999-04-06T04:00:00Z,951-85-8091,67641
5000009,Trenton,Rogelio,Prangle,Eye Candy,1980-05-18T04:00:00Z,922-19-4866,49319
5000010,Brock,Bruno,Perdue,Eye Candy,1967-07-18T04:00:00Z,968-59-2693,55465


In [0]:
%sql
select * from people_10m where gender = 'Superhero' limit 20;

id,firstName,middleName,lastName,gender,birthDate,ssn,salary
1,Pennie,Carry,Hirschmann,Superhero,1955-07-02T04:00:00Z,981-43-9345,56172
2,An,Amira,Cowper,Superhero,1992-02-08T05:00:00Z,978-97-8086,40203
3,Quyen,Marlen,Dome,Superhero,1970-10-11T04:00:00Z,957-57-8246,53417
4,Coralie,Antonina,Marshal,Superhero,1990-04-11T04:00:00Z,963-39-4885,94727
5,Terrie,Wava,Bonar,Superhero,1980-01-16T05:00:00Z,964-49-8051,79908
6,Chassidy,Concepcion,Bourthouloume,Superhero,1990-11-24T05:00:00Z,954-59-9172,64652
7,Geri,Tambra,Mosby,Superhero,1970-12-19T05:00:00Z,968-16-4020,38195
8,Patria,Nancy,Arstall,Superhero,1985-01-02T05:00:00Z,984-76-3770,102053
9,Terese,Alfredia,Tocque,Superhero,1967-11-17T05:00:00Z,967-48-7309,91294
10,Wava,Lyndsey,Jeandon,Superhero,1963-12-30T05:00:00Z,997-82-2946,56521


## Delete from a table

In [0]:
%sql
SELECT * FROM people_10m WHERE firstName IN ('Spiderman', 'Batman', 'Wonder', 'Thor', 'Hulk');

id,firstName,middleName,lastName,gender,birthDate,ssn,salary
9999998,Hulk,Bruce,Banner,Eye Candy,1992-09-17T04:00:00Z,953-38-9452,55250
9999999,Batman,Bruce,Wayne,Eye Candy,1984-05-22T04:00:00Z,906-51-2137,48500
10000000,Thor,Chris,Hemsworth,Eye Candy,1968-07-22T04:00:00Z,988-61-6247,90000
20000001,Spiderman,,Parker,Eye Candy,null,345-67-8901,55500
20000003,Wonder,,Woman,Superhero,null,567-89-0123,89900


In [0]:
%sql
SELECT * FROM people_10m WHERE firstName ='Spiderman'

id,firstName,middleName,lastName,gender,birthDate,ssn,salary
20000001,Spiderman,,Parker,Eye Candy,null,345-67-8901,55500


In [0]:
%sql
delete from people_10m where firstName = 'Spiderman'

num_affected_rows
1


In [0]:
%sql
SELECT * FROM people_10m WHERE firstName ='Spiderman'

id,firstName,middleName,lastName,gender,birthDate,ssn,salary


## Display table history

In [0]:
%sql
describe history people_10m

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
12,2024-01-27T17:47:46Z,5227933319595746,vtate01@qub.ac.uk,DELETE,"Map(predicate -> [""(firstName#14337 = Spiderman)""])",null,List(4277377919892141),0127-151556-vqpjky7e,11,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 2567, numCopiedRows -> 4, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 4224, numDeletionVectorsUpdated -> 0, numDeletedRows -> 1, scanTimeMs -> 3000, numAddedFiles -> 1, numAddedBytes -> 2519, rewriteTimeMs -> 1224)",null,Databricks-Runtime/14.2.x-scala2.12
11,2024-01-27T17:26:17Z,5227933319595746,vtate01@qub.ac.uk,UPDATE,"Map(predicate -> [""(gender#12714 = Male)""])",null,List(4277377919892141),0127-151556-vqpjky7e,10,WriteSerializable,false,"Map(numRemovedFiles -> 6, numRemovedBytes -> 147267517, numCopiedRows -> 1420479, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 46414, numDeletionVectorsUpdated -> 0, scanTimeMs -> 3232, numAddedFiles -> 6, numUpdatedRows -> 4812700, numAddedBytes -> 147270083, rewriteTimeMs -> 43182)",null,Databricks-Runtime/14.2.x-scala2.12
10,2024-01-27T17:25:27Z,5227933319595746,vtate01@qub.ac.uk,UPDATE,"Map(predicate -> [""(gender#11947 = Female)""])",null,List(4277377919892141),0127-151556-vqpjky7e,9,WriteSerializable,false,"Map(numRemovedFiles -> 6, numRemovedBytes -> 147747087, numCopiedRows -> 1021495, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 46360, numDeletionVectorsUpdated -> 0, scanTimeMs -> 2937, numAddedFiles -> 6, numUpdatedRows -> 5187303, numAddedBytes -> 147748734, rewriteTimeMs -> 43423)",null,Databricks-Runtime/14.2.x-scala2.12
9,2024-01-27T17:20:37Z,5227933319595746,vtate01@qub.ac.uk,UPDATE,"Map(predicate -> [""(gender#11141 = M)""])",null,List(4277377919892141),0127-151556-vqpjky7e,8,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 1761, numDeletionVectorsUpdated -> 0, scanTimeMs -> 1760, numAddedFiles -> 0, numUpdatedRows -> 0, numAddedBytes -> 0, rewriteTimeMs -> 0)",null,Databricks-Runtime/14.2.x-scala2.12
8,2024-01-27T17:20:32Z,5227933319595746,vtate01@qub.ac.uk,UPDATE,"Map(predicate -> [""(gender#10621 = F)""])",null,List(4277377919892141),0127-151556-vqpjky7e,7,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 953, numDeletionVectorsUpdated -> 0, scanTimeMs -> 952, numAddedFiles -> 0, numUpdatedRows -> 0, numAddedBytes -> 0, rewriteTimeMs -> 0)",null,Databricks-Runtime/14.2.x-scala2.12
7,2024-01-27T17:20:01Z,5227933319595746,vtate01@qub.ac.uk,UPDATE,"Map(predicate -> [""(gender#10098 = M)""])",null,List(4277377919892141),0127-151556-vqpjky7e,6,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 1637, numDeletionVectorsUpdated -> 0, scanTimeMs -> 1637, numAddedFiles -> 0, numUpdatedRows -> 0, numAddedBytes -> 0, rewriteTimeMs -> 0)",null,Databricks-Runtime/14.2.x-scala2.12
6,2024-01-27T17:19:57Z,5227933319595746,vtate01@qub.ac.uk,UPDATE,"Map(predicate -> [""(gender#9578 = F)""])",null,List(4277377919892141),0127-151556-vqpjky7e,5,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 886, numDeletionVectorsUpdated -> 0, scanTimeMs -> 883, numAddedFiles -> 0, numUpdatedRows -> 0, numAddedBytes -> 0, rewri

# ACID (time travel) With VERSION
## Query an earlier version of the table (time travel)

In [0]:
%sql
SELECT * FROM people_10m VERSION AS OF 0 limit 10

id,firstName,middleName,lastName,gender,birthDate,ssn,salary
5016568,Enrique,Emmett,Carvil,M,1960-01-17T05:00:00Z,909-88-7612,62241
5016569,Jordan,Alva,Penk,M,1979-05-02T04:00:00Z,936-39-5888,74778
5016570,Leo,Merlin,Conkay,M,1981-05-21T04:00:00Z,919-71-2948,82321
5016571,Bernard,Wiley,Thackham,M,1983-10-12T04:00:00Z,976-38-5505,72797
5016572,Devin,Loyd,Gipp,M,1990-01-20T05:00:00Z,934-44-9546,99538
5016573,Howard,Wade,Cokly,M,1979-04-10T05:00:00Z,901-17-5955,39998
5016574,Clyde,Anibal,Silvester,M,1964-02-03T05:00:00Z,992-74-4926,88749
5016575,Craig,Doug,Dohmer,M,1971-10-28T04:00:00Z,943-46-5612,75135
5016576,Eddie,Mikel,Handyside,M,1962-01-07T05:00:00Z,916-66-9845,67990
5016577,Burton,Scotty,Broggini,M,1968-02-16T05:00:00Z,973-55-2258,64515


# Optimise a table
### Once you have performed multiple changes to a table, you might have a lot of small files. To improve the speed of read queries, you can use OPTIMISE to collapse small files into larger ones:

In [0]:
%sql
optimize people_10m

path,metrics
dbfs:/user/hive/warehouse/people_10m,"List(1, 10, List(236792941, 236792941, 2.36792941E8, 1, 236792941), List(2281, 30171404, 2.3693455E7, 10, 236934550), 0, null, 1, 10, 0, true, 0, 0, 1706380055599, 1706380137578, 8, 1, null, List(0, 0), 8, 8, 74770, 0, null)"


## Z-ordering By Columns
#### [Useful read](https://docs.databricks.com/en/delta/optimize.html#how-often-should-i-run-optimize)
#### Z-Ordering is a technique used in Delta Lake to co-locate related information in the same set of files, which is automatically used by Delta Lake in data-skipping algorithms. 
This behavior dramatically reduces the amount of data that Delta Lake needs to read, resulting in faster queries and improved query performance.
By reorganising the data in storage, certain queries can read less data, so they run faster.


In [0]:
%sql
OPTIMIZE people_10m
ZORDER BY (gender)

path,metrics
dbfs:/user/hive/warehouse/people_10m,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 236792941), 0, List(0, 0), 0, null), 0, 1, 1, false, 0, 0, 1706380156515, 1706380159613, 8, 0, null, List(0, 0), 8, 8, 0, 0, null)"
